In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
import sys
sys.path.append('../Task 1/')
from efficient_apriori import apriori
from improved_apriori import Improved_Apriori
import json
import time
import os
import itertools
import ast
from tqdm import tqdm
import collections
import math
import random

In [2]:
# Process the dataset by chunks into tid and the list of bakerys the sales watched
def process_bakery_chunk(df, carry_over):
    if carry_over is not None:
        df = pd.concat([carry_over, df])
    groups = df.groupby('tid')['article'].apply(list)
    last_sales = df.iloc[-1]['tid']
    if last_sales in groups:
        carry_over = df[df['tid'] == last_sales]
        groups = groups.drop(last_sales)
    else:
        carry_over = None
    return groups, carry_over

In [3]:
carry_over = None
chunksize = 100000 # adjust this value depending on your available memory
# Might have to figure out a way to shuffle the dataset 
if(not os.path.exists('processed_bakery_output.txt')):
    with open('processed_bakery_output.txt', 'w') as f:
        for chunk in pd.read_csv('Bakery_sales.csv', chunksize=chunksize):
            groups, carry_over = process_bakery_chunk(chunk, carry_over)
            for sales, bakery_list in groups.items():
                f.write(f'{sales} {bakery_list}\n')

        # don't forget to process the last carry_over
        if carry_over is not None:
            groups, _ = process_bakery_chunk(carry_over, None)
            for sales, bakery_list in groups.items():
                f.write(f'{sales} {bakery_list}\n')

In [4]:
# Shuffle the text in chunks 
def shuffle_large_file(file_name, output_file_name, chunk_size):
    with open(file_name, 'r') as f:
        while True:
            lines = list(itertools.islice(f, chunk_size))
            if not lines:
                break
            random.shuffle(lines)
            with open(output_file_name, 'a') as out:
                out.write(''.join(lines))


# Call the function with your parameters
if(not os.path.exists('processed_bakery_output_shuffled.txt')):
    shuffle_large_file('processed_bakery_output.txt', 'processed_bakery_output_shuffled.txt', 3000000)

In [5]:
def read_file_in_partitions(file_path, partition_size):
    with open(file_path, 'r') as file:
        partition = []
        for line in file:
            partition.append(line)
            if len(partition) >= partition_size:
                yield partition
                partition = []
        if partition:  # yield any remaining lines
            yield partition

In [6]:
# Global variable to get the counts of all itemsets
global_counts = {}
def generate_global_counts(partition, global_candidates):

    # For 1th itemset, generate the transaction id list for the ith partition 
    transaction_id_dict = collections.defaultdict(list)
    for transaction_id in partition:
        for item in partition[transaction_id]:
            item_tuple = (item,)
            transaction_id_dict[item_tuple].append(transaction_id)

    # Filter based on the global candidates formed
    transaction_ids_dict = {item: transaction_ids for item, transaction_ids in transaction_id_dict.items() if item in global_candidates[1]}

    # Get the global count of all 1th itemset
    for item in transaction_id_dict:
        if(len(item) not in global_counts):
            global_counts[len(item)] = {}
        if(item not in global_counts[len(item)]):
            global_counts[len(item)][item] = len(transaction_id_dict[item])
        else:
            global_counts[len(item)][item] += len(transaction_id_dict[item])

    # Extend to find global count of all nth itemset from the global candidates
    for i in tqdm(range(1, len(global_candidates))):
        for itemset in global_candidates[i+1]:
            transaction_ids = set(transaction_id_dict[(itemset[0],)])
            for i in range(1, len(itemset)):
                # We are only interested in the transactions where all items in itemset is present
                transaction_ids = transaction_ids.intersection(set(transaction_ids_dict.get((itemset[i],), {})))
            if(len(itemset) not in global_counts):
                global_counts[len(itemset)] = {}

            if(itemset not in global_counts[len(itemset)]):
                global_counts[len(itemset)][itemset] = len(transaction_ids)
            else:
                global_counts[len(itemset)][itemset] += len(transaction_ids)



In [7]:
file_path = 'processed_bakery_output_shuffled.txt'
size_of_data = sum(1 for line in open(file_path))
min_support=0.001
partition_size = 3000
num_partitions = size_of_data// partition_size
print(num_partitions)
partition_candidates = []
global_candidates = collections.defaultdict(list)
# Step 1: Partitioning
global_min_support = math.ceil((min_support*size_of_data)/num_partitions)
for i, partition in enumerate(read_file_in_partitions(file_path, partition_size)):
    print(f'Partition {i+1}:')
    dict_bakery = {}
    for line in partition:
        sales, bakery_list_str = line.strip().split(' ', 1)
        bakery_list = ast.literal_eval(bakery_list_str)
        dict_bakery[sales] = bakery_list
   
    improved_apriori = Improved_Apriori(dict_bakery, min_support=min_support, min_confidence=1, verbose=2)
    # Step 2: Retreieve frequent itemset per partition
    partition_frequent_itemset = improved_apriori.apriori()
    # Efficient Apriori for sanity check
    # partition_frequent_itemset, _ = apriori(list(dict_bakery.values()), min_support = min_support, verbosity=2)

    # Form the global candidate set from the large itemset in each partitions
    # In this space, we ignore the count of itemset in each partition as they are not useful in our global support count
    # All they do is just show the itemset was large enough in the current partition
    # Merging Phase
    for level, itemset in partition_frequent_itemset.items():
        for key in itemset.keys():
            if(key not in global_candidates[level]):
                global_candidates[level].append(key)

45
Partition 1:
Found 95 candidate itemsets from 1st Level
Found 78 frequent itemsets from 1th item candidate sets
Found 3003 candidates for 2th item candidate sets
Time taken to find 2th item candidate sets: 0.0002200603485107422


100%|██████████| 3003/3003 [00:00<00:00, 146802.35it/s]


Found 214 frequent itemsets from 2th item candidate sets
Found 402 candidates for 3th item candidate sets
Time taken to find 3th item candidate sets: 0.008806228637695312


100%|██████████| 402/402 [00:00<00:00, 18139.77it/s]


Found 79 frequent itemsets from 3th item candidate sets
Found 13 candidates for 4th item candidate sets
Time taken to find 4th item candidate sets: 0.0007047653198242188


...

Found 95 frequent itemsets from 3th item candidate sets
Found 15 candidates for 4th item candidate sets
Time taken to find 4th item candidate sets: 0.001544952392578125


100%|██████████| 15/15 [00:00<00:00, 35444.82it/s]


Found 13 frequent itemsets from 4th item candidate sets
Found 0 candidates for 5th item candidate sets
Time taken to find 5th item candidate sets: 0.00011682510375976562


0it [00:00, ?it/s]

Found 0 frequent itemsets from 5th item candidate sets


In [8]:
# Now we have to read the lines in chunks for our disk-based operations
min_support_count = min_support * size_of_data
for i, partition in enumerate(read_file_in_partitions(file_path, partition_size)):
    print(f'Partition {i+1}:')

    # Hold the partition data in main memory
    dict_bakery = {}
    for line in partition:
        sales, bakery_list_str = line.strip().split(' ', 1)
        bakery_list = ast.literal_eval(bakery_list_str)
        dict_bakery[sales] = bakery_list
    # Pure disk based implementation would probably require us to save the global candidates in disk 
    generate_global_counts(dict_bakery, global_candidates)
    
        

Partition 1:


100%|██████████| 4/4 [00:00<00:00, 129.58it/s]


Partition 2:


100%|██████████| 4/4 [00:00<00:00, 130.64it/s]

Partition 3:


...

100%|██████████| 4/4 [00:00<00:00, 121.46it/s]


Partition 45:


100%|██████████| 4/4 [00:00<00:00, 123.64it/s]


Partition 46:


100%|██████████| 4/4 [00:00<00:00, 339.70it/s]


In [9]:
global_frequent_itemsets = {level: {itemset: count for itemset, count in itemsets.items() if count >= min_support_count} for level, itemsets in global_counts.items()}
global_frequent_itemsets

{1: {('BAGUETTE',): 15292,
  ('TRADITIONAL BAGUETTE',): 67688,
  ('CAMPAGNE',): 3905,
  ('CEREAL BAGUETTE',): 4961,
  ('BANETTE',): 15130,
  ('PAIN AU CHOCOLAT',): 10578,
  ('CROISSANT',): 11508,
  ('BANETTINE',): 2817,
  ('BOULE 200G',): 2691,
  ('COMPLET',): 3140,
  ('COUPE',): 20470,
  ('DIVERS VIENNOISERIE',): 827,
  ('ECLAIR',): 2006,
  ('FICELLE',): 2655,
  ('GRAND FAR BRETON',): 1378,
  ('QUIM BREAD',): 1304,
  ('BRIOCHE',): 1657,
  ('TARTELETTE',): 2861,
  ('NANTAIS',): 390,
  ('FLAN ABRICOT',): 568,
  ('FORMULE SANDWICH',): 4202,
  ('CAFE OU EAU',): 1436,
  ('MOISSON',): 3107,
  ('SPECIAL BREAD',): 5195,
  ('KOUIGN AMANN',): 1332,
  ('SUCETTE',): 525,
  ('GD KOUIGN AMANN',): 832,
  ('DIVERS BOULANGERIE',): 316,
  ('PAIN',): 1918,
  ('VIK BREAD',): 3147,
  ('SPECIAL BREAD KG',): 460,
  ('SAVARIN',): 337,
  ('PLAT 7.60E',): 244,
  ('PAIN BANETTE',): 2727,
  ('NOIX JAPONAISE',): 334,
  ('BOULE POLKA',): 502,
  ('SAND JB EMMENTAL',): 1505,
  ('BOULE 400G',): 4099,
  ('TRAITEUR',):

In [18]:
# Calculate global_frequent_itemsets (as already in your code)
# This part of the code remains unchanged

# Convert tuple keys to strings for the entire global_frequent_itemsets
global_frequent_itemsets_str = {}
for level, itemsets in global_frequent_itemsets.items():
    level_str = str(level)
    itemsets_str = {str(itemset): count for itemset, count in itemsets.items()}
    global_frequent_itemsets_str[level_str] = itemsets_str

# Save the converted dictionary to a JSON file
import json

# Define the file path for the JSON output
json_output_file = 'frequent_itemsets.json'

# Serialize and save the frequent itemsets to the JSON file
with open(json_output_file, 'w') as json_file:
    json.dump(global_frequent_itemsets_str, json_file)


In [ ]:
# Calculate global_frequent_itemsets (as already in your code)
# This part of the code remains unchanged

# Define the file path for the text output
text_output_file = 'frequent_itemsets.txt'

# Open the text file for writing
with open(text_output_file, 'w') as text_file:
    for level, itemsets in global_frequent_itemsets.items():
        text_file.write(f"Level {level}:\n")
        for itemset, count in itemsets.items():
            text_file.write(f"{itemset}: {count}\n")
        text_file.write("\n")

# Close the text file
text_file.close()


In [10]:
for level in global_frequent_itemsets:
    print(len(global_frequent_itemsets[level]))

73
178
50
3
0


In [12]:
data = pd.read_csv('Bakery_sales.csv')
data = data[['tid', 'article']]
grouped_data = data.groupby('tid')['article'].apply(list)
grouped_data = grouped_data.to_dict()

In [13]:
improved_apriori = Improved_Apriori(grouped_data, min_support=min_support, min_confidence=1, verbose=2)
frequent_bakery_set = improved_apriori.apriori()

Found 149 candidate itemsets from 1st Level
Found 73 frequent itemsets from 1th item candidate sets
Found 2628 candidates for 2th item candidate sets
Time taken to find 2th item candidate sets: 0.004758119583129883


100%|██████████| 2628/2628 [00:00<00:00, 3632.79it/s]


Found 178 frequent itemsets from 2th item candidate sets
Found 296 candidates for 3th item candidate sets
Time taken to find 3th item candidate sets: 0.004589080810546875


100%|██████████| 296/296 [00:00<00:00, 537.23it/s]


Found 50 frequent itemsets from 3th item candidate sets
Found 5 candidates for 4th item candidate sets
Time taken to find 4th item candidate sets: 0.0002620220184326172


100%|██████████| 5/5 [00:00<00:00, 277.10it/s]


Found 3 frequent itemsets from 4th item candidate sets
Found 0 candidates for 5th item candidate sets
Time taken to find 5th item candidate sets: 6.604194641113281e-05


0it [00:00, ?it/s]

Found 0 frequent itemsets from 5th item candidate sets


In [14]:
itemset, _ = apriori(list(grouped_data.values()), min_support = min_support, verbosity=2)

Generating itemsets.
 Counting itemsets of length 1.
  Found 149 candidate itemsets of length 1.
  Found 73 large itemsets of length 1.
    [('BAGUETTE',), ('PAIN AU CHOCOLAT',), ('PAIN',), ('TRADITIONAL BAGUETTE',), ('CROISSANT',), ('BANETTE',), ('BANETTINE',), ('SPECIAL BREAD',), ('COUPE',), ('SAND JB EMMENTAL',), ('KOUIGN AMANN',), ('BOULE 200G',), ('BOULE 400G',), ('CAMPAGNE',), ('MOISSON',), ('CAFE OU EAU',), ('BRIOCHE',), ('CEREAL BAGUETTE',), ('SEIGLE',), ('COMPLET',), ('DIVERS PATISSERIE',), ('GAL FRANGIPANE 4P',), ('COOKIE',), ('FICELLE',), ('PAIN AUX RAISINS',), ('FINANCIER X5',), ('VIK BREAD',), ('DIVERS VIENNOISERIE',), ('SANDWICH COMPLET',), ('PAIN BANETTE',), ('GRAND FAR BRETON',), ('QUIM BREAD',), ('SPECIAL BREAD KG',), ('GD KOUIGN AMANN',), ('BOULE POLKA',), ('DEMI BAGUETTE',), ('CHAUSSON AUX POMMES',), ('BAGUETTE GRAINE',), ('DIVERS CONFISERIE',), ('SUCETTE',), ('DIVERS BOULANGERIE',), ('BOISSON 33CL',), ('FORMULE SANDWICH',), ('DIVERS SANDWICHS',), ('CROISSANT AMANDES

In [15]:
frequent_bakery_set

{1: {('BAGUETTE',): 15273,
  ('BAGUETTE GRAINE',): 1500,
  ('BANETTE',): 15107,
  ('BANETTINE',): 2815,
  ('BOISSON 33CL',): 1468,
  ('BOULE 200G',): 2685,
  ('BOULE 400G',): 4082,
  ('BOULE POLKA',): 502,
  ('BRIOCHE',): 1656,
  ('CAFE OU EAU',): 1429,
  ('CAMPAGNE',): 3892,
  ('CEREAL BAGUETTE',): 4950,
  ('CHAUSSON AUX POMMES',): 1440,
  ('CHOU CHANTILLY',): 205,
  ('COMPLET',): 3133,
  ('COOKIE',): 1989,
  ('COUPE',): 19424,
  ('CROISSANT',): 11446,
  ('CROISSANT AMANDES',): 1802,
  ('DEMI BAGUETTE',): 1144,
  ('DEMI PAIN',): 163,
  ('DIVERS BOULANGERIE',): 311,
  ('DIVERS CONFISERIE',): 471,
  ('DIVERS PATISSERIE',): 480,
  ('DIVERS SANDWICHS',): 234,
  ('DIVERS VIENNOISERIE',): 818,
  ('ECLAIR',): 2002,
  ('FICELLE',): 2644,
  ('FINANCIER X5',): 1017,
  ('FLAN',): 830,
  ('FLAN ABRICOT',): 568,
  ('FONDANT CHOCOLAT',): 218,
  ('FORMULE SANDWICH',): 4180,
  ('FRAISIER',): 355,
  ('GAL FRANGIPANE 4P',): 203,
  ('GD FAR BRETON',): 161,
  ('GD KOUIGN AMANN',): 832,
  ('GRAND FAR BRET

In [16]:
itemset

{1: {('BAGUETTE',): 15273,
  ('PAIN AU CHOCOLAT',): 10529,
  ('PAIN',): 1917,
  ('TRADITIONAL BAGUETTE',): 67535,
  ('CROISSANT',): 11446,
  ('BANETTE',): 15107,
  ('BANETTINE',): 2815,
  ('SPECIAL BREAD',): 5182,
  ('COUPE',): 19424,
  ('SAND JB EMMENTAL',): 1502,
  ('KOUIGN AMANN',): 1330,
  ('BOULE 200G',): 2685,
  ('BOULE 400G',): 4082,
  ('CAMPAGNE',): 3892,
  ('MOISSON',): 3095,
  ('CAFE OU EAU',): 1429,
  ('BRIOCHE',): 1656,
  ('CEREAL BAGUETTE',): 4950,
  ('SEIGLE',): 1325,
  ('COMPLET',): 3133,
  ('DIVERS PATISSERIE',): 480,
  ('GAL FRANGIPANE 4P',): 203,
  ('COOKIE',): 1989,
  ('FICELLE',): 2644,
  ('PAIN AUX RAISINS',): 1986,
  ('FINANCIER X5',): 1017,
  ('VIK BREAD',): 3126,
  ('DIVERS VIENNOISERIE',): 818,
  ('SANDWICH COMPLET',): 2205,
  ('PAIN BANETTE',): 2725,
  ('GRAND FAR BRETON',): 1378,
  ('QUIM BREAD',): 1302,
  ('SPECIAL BREAD KG',): 460,
  ('GD KOUIGN AMANN',): 832,
  ('BOULE POLKA',): 502,
  ('DEMI BAGUETTE',): 1144,
  ('CHAUSSON AUX POMMES',): 1440,
  ('BAGUETT